In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from pathlib import Path
from sloyka import Geocoder,TextClassifiers, City_services

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thebe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2024-06-02 23:13:34,486 SequenceTagger predicts: Dictionary with 7 tags: O, S-Service, B-Service, E-Service, I-Service, <START>, <STOP>


In [17]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

from sloyka.src.data_getter import VKParser

In [22]:
domain = 'newstucentr'
token = '6fddc0e06fddc0e06fddc0e01e6cc5424166fdd6fddc0e0098175a5fbe443fba5b3ffe8'

In [23]:
df = VKParser().run_parser(domain, token, step=100, cutoff_date='2024-03-20')

2024-05-18 13:06:47
2024-05-04 17:15:46
2024-04-18 09:31:08
2024-03-31 14:00:23
2024-03-17 13:40:00
posts downloaded


100%|██████████| 410/410 [05:03<00:00,  1.35it/s]


comments downloaded


In [24]:
df.to_csv('..\\sloyka\\sample_data\\test_activity_data.csv', sep=';', encoding='utf-8')

In [2]:
df = pd.read_csv('..\\sloyka\\sample_data\\test_activity_data.csv', sep=';')

In [10]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [11]:
g = Geocoder()

2024-06-02 23:24:55,946 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS


In [12]:
gdf = g.run(osm_id = 338635, tags = {"admin_level": ["8"]}, date = "2024-04-22T00:00:00Z", df=df, text_column='text')

06-02 23:24 | INFO     | run_OSM_dfs started


C:\Projects\ITMO\sloyka\sloyka\src\geocoder.py:310: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  green_obj = ox.geometries_from_place(osm_city_name, tags)
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-B5Yiy4FA-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-B5Yiy4FA-py3.11\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_type

06-02 23:25 | INFO     | find_other_geo_obj started


c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-B5Yiy4FA-py3.11\Lib\site-packages\pandas\core\algorithms.py:522: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps_array.dtype], [])


TypeError: expected string or bytes-like object, got 'float'

In [ ]:
gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,...,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag
0,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большую,площадь,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большой проспект Санкт-Петербург Россия,Большой проспект,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street
1,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большую,площадь,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большой проспект Санкт-Петербург Россия,Большая аллея,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street
2,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большую,площадь,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большой проспект Санкт-Петербург Россия,Большая улица,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street
3,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большую,площадь,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большая аллея Санкт-Петербург Россия,Большой проспект,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street
4,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большую,площадь,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большая аллея Санкт-Петербург Россия,Большая аллея,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,2024-03-22 18:24:54,57489,Очень нужны места для купания и прогулок у вод...,NaN,0,NaN,comment,NaN,16164672.0,57475.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park
382,2024-03-18 19:05:54,57410,"[id973044|Екатерина], в прошлом году было боль...",NaN,0,NaN,reply,NaN,2585913.0,57322.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park
383,2024-03-17 15:43:39,57346,А где находится Некрасовский сад?,NaN,0,NaN,comment,NaN,7038519.0,57317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park
384,2024-03-18 07:47:56,57383,"[id7038519|Светлана Белованова], здравствуйте!...",NaN,0,NaN,reply,NaN,-143265175.0,57317.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park


In [10]:
gdf['text'].iloc[0]

'Дорого и плохо, а порой еще и опасно  Сюжет «78» о некачественном и дорогом благоустройстве в Петербурге. В этот раз журналисты решили уделить особое внимание нашей местной компании ООО ПСФ «Корт» (и аффилированному с ней ООО «Зеленстрой»).  Её учредителем по документам является бывшая жена Виктора Шабнова, экс-зампредседателя Комитета по благоустройству, уволенного из Смольного в 2018 г. по статье «утрата доверия» за нарушение закона о противодействии коррупции.  Несмотря на такую опалу, семейный бизнес Шабновых вовсю процветает - в последние годы они, без какой-либо конкуренции и снижения цены, забирают все самые жирные контракты по благоустройству в южных районах Петербурга.  Парк Авиаторов (почти 1 млрд рублей), сквер Раисы Штрейс, скейт-парк и [https://vk.com/wall-33468323_41130|«собачий аквапарк»] на Железнодорожной в Пушкине за 78 млн (о котором также упомянуто в этом сюжете), I этап благоустройства «большого круга» в Славянке за 95 млн - список можно бесконечно продолжать. Тол

In [28]:
text_classifier = TextClassifiers('Sandrro/text_to_subfunction_v10')

In [30]:
gdf[['cats', 'probs']] =  gdf["text"].progress_map(lambda x: text_classifier.run_text_classifier(x)).to_list()

  0%|          | 0/386 [00:00<?, ?it/s]c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-B5Yiy4FA-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  1%|          | 2/386 [00:01<05:28,  1.17it/s]c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-B5Yiy4FA-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  1%|          | 3/386 [00:03<07:33,  1.18s/it]c:\Users\thebe\AppData\Local\pypoetry\Cache\virtualenvs\sloyka-B5Yiy4FA-py3.11\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated an

In [31]:
gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,...,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag,cats,probs
0,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большой проспект Санкт-Петербург Россия,Большой проспект,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street,Жалобы на незаконную торговлю,1.0
1,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большой проспект Санкт-Петербург Россия,Большая аллея,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street,Жалобы на незаконную торговлю,1.0
2,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большой проспект Санкт-Петербург Россия,Большая улица,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street,Жалобы на незаконную торговлю,1.0
3,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большая аллея Санкт-Петербург Россия,Большой проспект,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0
4,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,"Большой проспект Санкт-Петербург Россия,Больш...","['Большой проспект Санкт-Петербург Россия', '...",Большая аллея Санкт-Петербург Россия,Большая аллея,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,2024-03-22 18:24:54,57489,Очень нужны места для купания и прогулок у вод...,NaN,0,NaN,comment,NaN,16164672.0,57475.0,...,NaN,NaN,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,unknown_Экология,1.0
382,2024-03-18 19:05:54,57410,"[id973044|Екатерина], в прошлом году было боль...",NaN,0,NaN,reply,NaN,2585913.0,57322.0,...,NaN,NaN,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,Жалобы на несвоевременный вывоз отходов,0.84
383,2024-03-17 15:43:39,57346,А где находится Некрасовский сад?,NaN,0,NaN,comment,NaN,7038519.0,57317.0,...,NaN,NaN,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,Зачисление в д/сад,0.709
384,2024-03-18 07:47:56,57383,"[id7038519|Светлана Белованова], здравствуйте!...",NaN,0,NaN,reply,NaN,-143265175.0,57317.0,...,NaN,NaN,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,unknown_Благоустройство,0.914


In [10]:
percentages = df.copy()['cats'].value_counts(normalize=True)[:5] * 100

In [11]:
percentages_df = pd.DataFrame({'Value': percentages.index, 'Percentage': percentages.values})

In [12]:
percentages_df

,Value,Percentage
0,Не ЦУР,20.338983
1,"Отсутствие информирования об изменении, закрыт...",8.474576
2,unknown_Благоустройство,6.779661
3,"Все другие вопросы строительства, не вошедшие ...",6.779661
4,unknown_Строительство,5.084746


In [10]:
df['toponym'] = 'улица'

In [11]:
user_partition = df['from_id'].loc[df['cats'] == 'Не ЦУР'].value_counts(normalize=True) * 100

In [12]:
user_partition = pd.DataFrame({'user_id': user_partition.index, 'Percentage': user_partition.values})

In [13]:
user_partition

,user_id,Percentage
0,1492921.0,20.0
1,967593.0,10.0
2,636.0,10.0
3,650393846.0,10.0
4,871865.0,10.0
5,185082.0,10.0
6,439008.0,10.0
7,632043.0,10.0
8,144340.0,10.0


In [32]:
gdf = City_services().run(gdf, 'text')

In [33]:
gdf

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,...,addr_to_geocode,only_full_street_name,Location,geometry,other_geo_obj,geo_obj_tag,cats,probs,City_services_extraced,City_services
0,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большой проспект Санкт-Петербург Россия,Большой проспект,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street,Жалобы на незаконную торговлю,1.0,[],[]
1,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большой проспект Санкт-Петербург Россия,Большая аллея,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street,Жалобы на незаконную торговлю,1.0,[],[]
2,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большой проспект Санкт-Петербург Россия,Большая улица,"Большой проспект, Репино, Санкт-Петербург, Сев...",POINT (29.85599 60.19519),NaN,street,Жалобы на незаконную торговлю,1.0,[],[]
3,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большая аллея Санкт-Петербург Россия,Большой проспект,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0,[],[]
4,2024-06-02 14:31:34,58907,В Центральном районе продолжается борьба с нез...,577.0,4,1.0,post,NaN,NaN,NaN,...,Большая аллея Санкт-Петербург Россия,Большая аллея,"Большая аллея, Острова, округ Чкаловское, Санк...",POINT (30.28604 59.97685),NaN,street,Жалобы на незаконную торговлю,1.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,2024-03-22 18:24:54,57489,Очень нужны места для купания и прогулок у вод...,NaN,0,NaN,comment,NaN,16164672.0,57475.0,...,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,unknown_Экология,1.0,[садах],[Сад]
382,2024-03-18 19:05:54,57410,"[id973044|Екатерина], в прошлом году было боль...",NaN,0,NaN,reply,NaN,2585913.0,57322.0,...,NaN,NaN,NaN,POINT (30.37323 59.94547),Таврический сад,park,Жалобы на несвоевременный вывоз отходов,0.84,[саду],[Сад]
383,2024-03-17 15:43:39,57346,А где находится Некрасовский сад?,NaN,0,NaN,comment,NaN,7038519.0,57317.0,...,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,Зачисление в д/сад,0.709,[сад],[Сад]
384,2024-03-18 07:47:56,57383,"[id7038519|Светлана Белованова], здравствуйте!...",NaN,0,NaN,reply,NaN,-143265175.0,57317.0,...,NaN,NaN,NaN,POINT (30.36747 59.93735),Некрасовский сад,park,unknown_Благоустройство,0.914,"[сад, сад, памятник, сада]","[Сад, Сад, Памятник, Сад]"


In [35]:
city_services = []
for x in gdf['City_services'].to_list():
    city_services.extend(x if isinstance(x, list) else [x])
    
city_services = set(city_services)

In [36]:
city_services

{'Аэродром',
 'Бар',
 'Библиотека',
 'Ботанический сад',
 'Газон',
 'Детская площадка',
 'Детский сад',
 'Залив',
 'Заповедник',
 'Зоопарк',
 'Кафе',
 'Лес',
 'Магазин',
 'Метро',
 'Музей',
 'Озеро',
 'Памятник',
 'Парк',
 'Парковка',
 'Поликлиника',
 'Продукты',
 'Река',
 'Роддом',
 'Сад',
 'Спортивная площадка',
 'Театр',
 'Торговый центр',
 'Университет',
 'Фонтан',
 'Цирк',
 'Школа'}

In [38]:
gdf['City_services'] = gdf['City_services'].apply(lambda x: ','.join(set(x)) if x else None)

In [39]:
gdf['City_services']

0              None
1              None
2              None
3              None
4              None
           ...     
381             Сад
382             Сад
383             Сад
384    Памятник,Сад
385    Памятник,Сад
Name: City_services, Length: 386, dtype: object

In [91]:
import numpy as np
df['toponym'] = np.random.randint(0, 3, df.shape[0])

In [92]:
df

,date,id,text,views.count,likes.count,reposts.count,type,link,from_id,post_id,parents_stack,group_name,cats,probs,toponym,City_services_extraced,City_services
0,2024-05-22 16:47:48,18024,"Дорого и плохо, а порой еще и опасно Сюжет «7...",1104.0,45,7.0,post,https://vk.com/wall-33468323_41130|«собачий,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.849,1,"[Корт, Парк, аквапарк, Корт, Корт, Корт]","Аквапарк,Морг,Парк"
1,2024-05-22 13:26:45,18023,26 МАЯ | Открытый детский турнир по хоккею Ув...,374.0,3,1.0,post,https://vk.com/mamo_shushary|муниципалитет],NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,0.406,0,[],None
2,2024-05-17 16:51:13,18011,Прямой эфир с [id1745844|Екатериной Краевой] с...,2302.0,26,8.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Благоустройство,1.0,2,[кладбища],Кладбище
3,2024-05-16 17:46:54,18003,Показательное отношение Милонова к своим избир...,2515.0,30,11.0,post,https://vk.com/wall-516730_675000|«помощь»],NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.991,2,[],None
4,2024-05-14 15:46:33,17992,Шушары Today,3022.0,23,1.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Жалобы на незаконную торговлю,0.896,1,[],None
5,2024-05-13 01:05:41,17977,"ВНИМАНИЕ!Сегодня, как обычно, без должного инф...",11194.0,26,34.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,unknown_Транспорт,0.956,2,[кладбища],Кладбище
6,2024-05-06 14:10:00,17964,В этом году за счёт муниципального бюджета Шуш...,3149.0,16,17.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,"Отсутствие информирования об изменении, закрыт...",0.989,1,[],None
7,2024-04-27 10:27:31,17937,Сегодня свой День рождения отмечает неотъемлем...,7119.0,213,26.0,post,NaN,NaN,NaN,NaN,Шумные Шушары,Не ЦУР,0.519,0,[],None
8,2024-05-22 13:55:20,18025,А что за движуха сегодня утром была на Новгоро...,NaN,3,NaN,comment,NaN,1502930.0,18024.0,NaN,Шумные Шушары,Критика транспортной реформы,0.688,1,[],None
9,2024-05-22 15:16:35,18026,"[id1502930|Анна], таки да, к нам точно едет ""р...",NaN,7,NaN,reply,NaN,967593.0,18024.0,18025.0,Шумные Шушары,Не ЦУР,0.955,2,[],None


In [96]:
router = {0: 'Первый', 1: 'Второй', 2: None}

In [97]:
df['toponym'] = df['toponym'].apply(lambda x: router[x])

In [41]:
top_toponyms = gdf['only_full_street_name'].value_counts(normalize=True) * 100

In [42]:
top_toponyms_names = top_toponyms.index

In [ ]:
top_toponymsdf = pd.DataFrame([top_toponyms_names, top_toponyms.to_list()]).T

In [45]:
top_toponyms_df[:5]

,0,1
0,Невский проспект,16.140351
1,Невский переулок,11.578947
2,Невский путепровод,11.578947
3,улица Чайковского,3.859649
4,Дворцовая улица,2.105263


In [36]:
toponyms = df['toponym'].unique()

In [143]:
toponyms_names = ['Второй']

In [144]:
for i in toponyms_names:
    
    posts_ids = df['id'].loc[df['toponym'] == i].to_list()
    comments_ids = df['id'].loc[df['post_id'].isin(posts_ids) & df['toponym'].isin([i, None])].to_list()
    replies_ids = df['id'].loc[df['parents_stack'].isin(comments_ids) & df['toponym'].isin([i, None])].to_list()
    
    all_ids = tuple(sorted(list(set(posts_ids + comments_ids + replies_ids))))
    
    toponym_df = df.loc[df['id'].isin(all_ids)]
    part_users = len(toponym_df['from_id'].unique())/len(df['from_id'].unique())
    part_messages = len(toponym_df['id'].unique())/len(df['id'].unique())
    
    services = df['City_services_extraced'].loc[df['toponym'] == i]

In [147]:
part_messages

0.4406779661016949

In [158]:
res = []
for i in services:
    res += i
    
res = set(res)

In [159]:
res

{'Корт', 'Парк', 'аквапарк', 'горы', 'кладбища'}

In [161]:
count_dict = {}

for i in res:
    count_dict[i] = 0

In [171]:
df['City_services_extraced'].iloc[0]

['Корт', 'Парк', 'аквапарк', 'Корт', 'Корт', 'Корт']

In [180]:
for i in range(len(df)):
    for j in set(df['City_services_extraced'].iloc[i]):
        if j in count_dict:
            count_dict[j] = count_dict[j] + 1

In [184]:
service_df_list = [[key, value] for key, value in count_dict.items()]

In [189]:
services_df = pd.DataFrame(service_df_list, columns=['service', 'count'])

In [191]:
services_df.sort_values(by='count', ascending=False)

,service,count
4,кладбища,5
0,горы,1
1,Парк,1
2,Корт,1
3,аквапарк,1


In [119]:
toponym_df = df.loc[df['id'].isin(all_ids)]
part_users = len(toponym_df['from_id'].unique())/len(df['from_id'].unique())
part_messages = len(toponym_df['id'].unique())/len(df['id'].unique())

In [120]:
part_users

0.47058823529411764

In [121]:
part_messages

0.3220338983050847